Upload as many files as You want.
Notebook will automatically create exclusive folders for them. 📂


In [ ]:
#@title <font color=gold>←</font> Install modules. **(Required for the first time usage!)**
%%capture
!rm -rf sample_data
!python -m pip install py7zr pydub

In [ ]:
#@title <font color=red>←</font> Remove <b>ALL</b> files. <font color=red><b>This action is irreversible!</font></b>
#@markdown - **Required to restart processing!**
%%capture
%cd /content
!rm -rf *

In [ ]:
#@title <font color=lime>←</font> Upload files and <b>Unisonize!</b>
import os
from py7zr import *
from pydub import *
from IPython.display import *
from datetime import datetime
from google.colab import files
from itertools import zip_longest

#---#

__author__ = "kubinka0505"
__credits__ = __author__
__version__ = "1.0"
__date__ = "07.02.2021"
 
#---#

Format = "WAV" #@param ["MP3", "OGG", "WAV", "FLAC"]
Invert_Phase_for_Even_Files = False  #@param {type:"boolean"}

#@markdown - After processing files will be packed to `Audio.7z` archive and **downloaded automatically**.

#---#

os.chdir("/content/")

__Time = "".join(list(filter(lambda s:s.isdigit(),str(datetime.now()).split(" ")[1].split(".")[0])))
ArchiveName = "Audio_{0}.7z".format(__Time)
AfterName = "_stereo"

if os.listdir() == [".config", ".ipynb_checkpoints"]: Files = files.upload()

Files = sorted([x for x in os.listdir() if x.endswith(".{0}".format(Format.lower()))], key = str)
Files_Sorted = [[y, x] for x, y in zip_longest(Files[1::2], Files[::2])]

# Create folder for each uploaded file
for _Files in Files_Sorted:
	FolderName = _Files[0].split(".")[0]
	os.mkdir(FolderName)
	os.rename(_Files[0], "{0}/{1}".format(FolderName, _Files[0]))
	os.rename(_Files[1], "{0}/{1}".format(FolderName, _Files[1]))

Folders = sorted(next(os.walk("."))[1], key = str)[2:]

# Modify each audio file in previously created folders
for Folder in Folders:
	os.chdir("/content/{0}".format(Folder))

	Files__ = os.listdir()

	Name = "{0}{1}.{2}".format(Files__[0].split(".")[0], AfterName, Format.lower())

	#---#

	print("\r", end = "Processing {0}...".format(Folder))
	Left = AudioSegment.from_file(Files__[0])
	Right = AudioSegment.from_file(Files__[1])
	if Invert_Phase_for_Even_Files == True: Right = Right.invert_phase()

	Left = Left.pan(-1)
	Right = Right.pan(1)

	Left = Left.normalize(headroom = 0)
	Right = Right.normalize(headroom = 0)

	Multi = Left.overlay(Right)
	Multi = Multi.normalize(headroom = 0)

	Multi.export(Name, format = Format)

	# Move modified file and remove uploaded files
	os.remove(Files__[0])
	os.remove(Files__[1])
	os.rename("{0}/{1}".format(os.getcwd(), Name), "{0}/{1}".format(os.path.realpath(".."), Name))
	os.rmdir(os.getcwd())
	os.chdir("..")

os.chdir("/content/")
with SevenZipFile(
	ArchiveName,
	"w",
	filters = [{"id":  FILTER_LZMA2, "preset":  PRESET_DEFAULT}]
	) as Archive:
		for x in os.listdir():
			if x.endswith(Format.lower()):
				Archive.write(x)

if len(os.listdir()) != 2: files.download(ArchiveName)